<a href="https://colab.research.google.com/github/SevilAnna/UniversityClasses/blob/master/EM%C3%9C312%20Class/EMU312_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from pandas.io.json import json_normalize 
import folium
from geopy.geocoders import Nominatim
print('Done!')

Done!


In [ ]:
# Hacettepe University's address details  #hacettepe: 39.8730 , 32.7369
address = 'Hacettepe University'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
h_latitude = location.latitude        
h_longitude = location.longitude
print("Hacettepe University's Latitude & Longitude are:", h_latitude, h_longitude)

Hacettepe University's Latitude & Longitude are: 39.867445950000004 32.73548807525388


In [ ]:
# Foursquare connection details
client_id = #redacted
client_secret = #redacted
version = '20180604' 
search_query = "Vet"
radius = 5000
LIMIT = 1000

url= 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    client_id, client_secret, h_latitude, h_longitude, version, search_query, radius, LIMIT)
print("Ok")

Ok


In [ ]:
results = requests.get(url).json()
venues = results['response']['venues']
# making our dataframe
df = pd.json_normalize(venues)
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,506fe3a0e4b018a97703b2de,Vet Complex,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",v-1622623571,False,Osmanağa Konakları 2494/3 Sok. No:7 Ümitköy,39.893361,32.701647,"[{'label': 'display', 'lat': 39.89336107000395...",4084,TR,Yenimahalle,Ankara,Ankara,Türkiye,"[Osmanağa Konakları 2494/3 Sok. No:7 Ümitköy, ...",NaN,NaN
1,51a9da89498e83fcddb1bd44,Bil-Vet Bilkent Veteriner Muayenehanesi,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",v-1622623571,False,NaN,39.880817,32.749900,"[{'label': 'display', 'lat': 39.88081657293824...",1931,TR,NaN,NaN,NaN,Türkiye,[Türkiye],NaN,NaN
2,4eaab7690cd6a81e7cb29d21,Hacettepe Üniversitesi Aile ve Tüketici Biliml...,"[{'id': '4bf58dd8d48988d19b941735', 'name': 'C...",v-1622623571,False,İktisadi ve İdari Bilimler Fakültesi,39.865251,32.735877,"[{'label': 'display', 'lat': 39.8652506260855,...",246,TR,NaN,Ankara,Ankara,Türkiye,[İktisadi ve İdari Bilimler Fakültesi (Beytepe...,Beytepe Kampüsü,06370
3,4e16b6c4ae60de5d28f536a0,Radyo ve Televizyon Üst Kurulu,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1622623571,False,Bilkent,39.884568,32.755457,"[{'label': 'display', 'lat': 39.88456763334146...",2557,TR,NaN,Ankara,Ankara,Türkiye,"[Bilkent, Ankara, Türkiye]",NaN,NaN
4,50ab3789e4b01ef7187cc8cd,HATÜT ( aile ve tüketici toplulugu odasi),"[{'id': '4bf58dd8d48988d174941735', 'name': 'C...",v-1622623571,False,NaN,39.865190,32.735339,"[{'label': 'display', 'lat': 39.86518964501122...",251,TR,NaN,NaN,NaN,Türkiye,[Türkiye],NaN,NaN


In [ ]:
columns_filtered = ['id', 'name', 'categories', 'location.address', 'location.lat', 'location.lng','location.distance']
df_filtered = df.loc[:, columns_filtered]
def write_category(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
df_filtered['categories'] = df_filtered.apply(write_category, axis=1)
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]
df_filtered.head()

,id,name,categories,address,lat,lng,distance
0,506fe3a0e4b018a97703b2de,Vet Complex,Veterinarian,Osmanağa Konakları 2494/3 Sok. No:7 Ümitköy,39.893361,32.701647,4084
1,51a9da89498e83fcddb1bd44,Bil-Vet Bilkent Veteriner Muayenehanesi,Veterinarian,NaN,39.880817,32.749900,1931
2,4eaab7690cd6a81e7cb29d21,Hacettepe Üniversitesi Aile ve Tüketici Biliml...,College Science Building,İktisadi ve İdari Bilimler Fakültesi,39.865251,32.735877,246
3,4e16b6c4ae60de5d28f536a0,Radyo ve Televizyon Üst Kurulu,Government Building,Bilkent,39.884568,32.755457,2557
4,50ab3789e4b01ef7187cc8cd,HATÜT ( aile ve tüketici toplulugu odasi),Coworking Space,NaN,39.865190,32.735339,251


In [ ]:
for item in df_filtered['categories']:
  if item != "Veterinarian":
    print(item,end=", ")

College Science Building, Government Building, Coworking Space, Government Building, University, College Lab, Volleyball Court, Residential Building (Apartment / Condo), Tech Startup, Cosmetics Shop, Tech Startup, College Classroom, Student Center, College Classroom, Auditorium, Medical Center, Government Building, High School, Coworking Space, Medical School, Hospital, Auditorium, Coworking Space, Hospital, Government Building, College Lab, Animal Shelter, Dog Run, Pet Service, College Academic Building, Tech Startup, Professional & Other Places, Building, University, Capitol Building, Research Laboratory, Office, 

In [ ]:
vet_list = ["Veterinarian","Pet Service","Dog Run","Animal Shelter","Medical Center"]
notvet_list = []
for item in df_filtered['categories']:
  if item not in vet_list:
    notvet_list.append(item)
for i in notvet_list:
  df_filtered = df_filtered[df_filtered.categories != i]
df_final = df_filtered.reset_index(drop=True)
df_final.head()

,id,name,categories,address,lat,lng,distance
0,506fe3a0e4b018a97703b2de,Vet Complex,Veterinarian,Osmanağa Konakları 2494/3 Sok. No:7 Ümitköy,39.893361,32.701647,4084
1,51a9da89498e83fcddb1bd44,Bil-Vet Bilkent Veteriner Muayenehanesi,Veterinarian,NaN,39.880817,32.749900,1931
2,5912f4534b78c50ab0d86dc9,Vetipedia Angora Veteriner Kliniği,Veterinarian,MUTLUKENT MAHALLESİ HİTİT BULVARI 12/A ÇANKAYA,39.870215,32.717270,1586
3,4f4a061be4b09620891a9442,Vetrium,Veterinarian,Orun Villalari No:3,39.872535,32.695164,3491
4,4e0f994f6365434360092f37,Beysukent Veteriner Kliniği,Medical Center,Beysukent,39.885225,32.715393,2619


In [ ]:
df_sorted = df_final.sort_values(by=['distance'])
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,id,name,categories,address,lat,lng,distance
0,5912f4534b78c50ab0d86dc9,Vetipedia Angora Veteriner Kliniği,Veterinarian,MUTLUKENT MAHALLESİ HİTİT BULVARI 12/A ÇANKAYA,39.870215,32.717270,1586
1,51a9da89498e83fcddb1bd44,Bil-Vet Bilkent Veteriner Muayenehanesi,Veterinarian,NaN,39.880817,32.749900,1931
2,5188fdfb498ec638ec479615,bilkent veteriner klinigi,Veterinarian,NaN,39.881772,32.749918,2015
3,5bfedbaffd9d73002c051663,Foks Veteriner Kliniği,Veterinarian,Beytepe Mahallesi Kanuni Sultan Süleyman Bulva...,39.855791,32.712353,2364
4,4e0f994f6365434360092f37,Beysukent Veteriner Kliniği,Medical Center,Beysukent,39.885225,32.715393,2619
5,5231d9bc11d2688bc8e8352b,Pet's Choice Veteriner,Veterinarian,NaN,39.894447,32.728823,3059
6,50d17621b0ed96b668459007,Active Veteriner Sağlık Merkezi,Pet Service,Ahmet Taner Kışlalı Mahallesi Beyler Caddesi 2...,39.865047,32.696331,3356
7,4f4a061be4b09620891a9442,Vetrium,Veterinarian,Orun Villalari No:3,39.872535,32.695164,3491
8,5f2f15187c8f6806c549af33,Vetclinic Hayvan Hastanesi,Veterinarian,Mutlukent Mahallesi,39.889367,32.703953,3634
9,534a6209498e435244ed900a,Pet's and Peace Veteriner Klinigi,Veterinarian,Ahmet Taner Kislali Mah. S.Saltoglu Bulv. No:3,39.873311,32.693214,3670


In [ ]:
vets_map = folium.Map(location=[h_latitude+0.013, h_longitude-0.02], zoom_start=13, width='50%', height='60%', max_zoom=16, min_zoom=11)

folium.Marker(
    [h_latitude, h_longitude], 
    popup="Hacettepe", 
    tooltip="Starting point",
    icon=folium.Icon(color='red',icon_color='white',prefix='fa',icon='graduation-cap')
).add_to(vets_map)

for lat, lng, label in zip(df_final.lat, df_final.lng, df_final.name):
    folium.Marker(
        [lat, lng],
        popup=label,
        tooltip="Click to see vet's name",
        icon=folium.Icon(color='darkpurple',icon_color='white',prefix='fa',icon='paw') 
    ).add_to(vets_map)

# icons from:   https://fontawesome.com/v5.15/icons?d=gallery&p=2&c=animals,education&m=free

vets_map

iyi tatiller :)